In [55]:
import torch as th
def low_rank_eigen(G, num_eig):
    """
    Calculate num_eig eigenvectors and eigenvalues of gaussian matrix G.
    Enables lower dimensional solving.
    """
    S, Q = th.linalg.eigh(G)
    eig_indices = th.flip(th.argsort(th.abs(S)), dims=(0, ))[:num_eig].tolist()
    Q = Q[:, eig_indices]  # eigenvectors
    S = S[eig_indices]  # eigenvalues.
    return Q, S

In [56]:
import numpy as np
def low_rank_eigen_np(G, num_eig):
    """
    Calculate num_eig eigenvectors and eigenvalues of gaussian matrix G.
    Enables lower dimensional solving.
    """
    S, Q = np.linalg.eigh(G)
    eig_indices = list(np.argsort(np.abs(S))[::-1][:num_eig])
    Q = Q[:, eig_indices]  # eigenvectors
    S = S[eig_indices]  # eigenvalues.
    return Q, S

In [68]:
a = th.rand(3,3,dtype=th.float64).cuda()
b = a.detach().cpu().numpy()
num_eig = 2
Q, S = low_rank_eigen(a, num_eig)
Q1, S1 = low_rank_eigen_np(b, num_eig)
print(np.isclose(Q.detach().cpu().numpy(), Q1).all())

True


In [99]:
c = th.rand(3,3,dtype=th.float64)
c1 = c.numpy()
d1 = np.all(np.linalg.eigvals(c1) > 0)
print(np.linalg.eigvals(c1))
print(th.linalg.eigvals(c))
d = th.all(th.real(th.linalg.eigvals(c))>0)
print(d1, d)

[1.34571985 0.01520165 0.67171602]
tensor([1.3457+0.j, 0.0152+0.j, 0.6717+0.j], dtype=torch.complex128)
True tensor(True)


In [129]:
PX = th.rand(5, 3)
muX = th.div(th.sum(PX, dim=0), 5)
X = th.rand(2, 3)
mux = th.tile(muX, (2, 1))
print(muX.shape)


torch.Size([3])


In [126]:
P1 = th.rand(5)
Y = th.rand(5, 3)
P = th.rand(5, 2)
X_hat = th.rand(2, 3)
muY = th.div(th.sum(th.mm(P.permute(1, 0), Y), dim=0), 5.)
M = Y.shape[0]
print(muY)
print(th.tile(muY, (M, 1)))
Y_hat = Y - th.tile(muY, (M, 1))

YPY = th.sum(th.mul(P1, th.sum(th.mul(Y_hat, Y_hat), dim=1)))
print(YPY)

tensor([0.6027, 0.3797, 0.1964])
tensor([[0.6027, 0.3797, 0.1964],
        [0.6027, 0.3797, 0.1964],
        [0.6027, 0.3797, 0.1964],
        [0.6027, 0.3797, 0.1964],
        [0.6027, 0.3797, 0.1964]])
tensor(0.1926)


In [127]:
A = th.mm(X_hat.permute(1, 0), P.permute(1, 0))
A = th.mm(A, Y_hat)
U, _, V = th.linalg.svd(A, full_matrices=True)
print(U.shape, V.shape)
C = th.ones((3, ))
C[3-1] = th.linalg.det(th.mm(U, V))
ee = np.linalg.det(np.dot(U.numpy(), V.numpy()))
print(C, ee)

torch.Size([3, 3]) torch.Size([3, 3])
tensor([ 1.0000,  1.0000, -1.0000]) -1.0000002


In [128]:
th.diag(C)

tensor([[ 1.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000],
        [ 0.0000,  0.0000, -1.0000]])

In [133]:
a = th.rand(3,3)
b = th.rand(3).reshape(-1, 1)
a1 = a.numpy()
b1 = b.numpy()
print(np.dot(a1,b1))

[[0.5566097 ]
 [0.20196609]
 [0.61216605]]


In [135]:
th.mm(a, b)

tensor([[0.5566],
        [0.2020],
        [0.6122]])

In [9]:
import numpy as np
import torch as th

X_hat = np.random.randn(20, 3)
th_X_hat = th.from_numpy(X_hat)
Pt1 = np.random.randn(20, 1)
th_Pt1 = th.from_numpy(Pt1)
print(Pt1.shape)
xPx = np.dot(np.transpose(Pt1), np.sum(np.multiply(X_hat, X_hat), axis=1))
xPx
 

(20, 1)


array([20.67719662])

In [33]:
B = th.eye(3).numpy()
print(B.shape)
YPY = 2 * th.ones(3,3).numpy()
print(YPY.shape)
trBYPYP = np.trace(np.dot(np.dot(B, YPY),B))
trBYPYP
#th.trace(th.mm(th.mm(B, YPY), B))

(3, 3)
(3, 3)


6.0

In [52]:
P = np.random.rand(20, 10)
thP = th.from_numpy(P)
PX = np.random.rand(20, 3)
thpx = th.from_numpy(PX)
Y = np.random.rand(20, 3)
thy = th.from_numpy(Y)
Np=1.5
B = np.eye(3)
thB = th.from_numpy(B)
muX = np.divide(np.sum(PX, axis=0), Np)
thmuX = th.div(th.sum(thpx, dim=0), Np)
thmuY = th.div(th.sum(th.mm(thP.permute(1, 0), thy), dim=0), Np)
muY = np.divide(np.sum(np.dot(np.transpose(P), Y), axis=0), Np)
print(np.transpose(muX) - np.dot(np.transpose(B), np.transpose(muY)))
print((thmuX.reshape(-1, 1) - th.mm(thB.permute(1, 0), thmuY.reshape(-1, 1))).permute(1,0))

[-27.94191251 -29.43326787 -28.66592013]
tensor([[-27.9419, -29.4333, -28.6659]], dtype=torch.float64)


In [53]:
th.eye(3)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])